In [1]:
import pandas as pd
import numpy as np
from scipy import stats

## Data Load

In [2]:
df = pd.read_csv('./data/BankChurners.csv')

## 편의를 위해 target 열 이탈고객의 경우 1, 유지 고객은 0으로 변경

In [3]:
df['Attrition_Flag'] = df['Attrition_Flag'].map({'Existing Customer': 0, 'Attrited Customer': 1})

## 필요없는 열 제거

In [4]:
df.drop(['CLIENTNUM','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1, inplace = True)

# 파생변수 생성

## 1. 거래 당 지불 금액

- Avg_Trans_Amt = Total_Trans_Amt/Total_Trans_Ct

In [5]:
df['Avg_Trans_Amt'] = df['Total_Trans_Amt']/df['Total_Trans_Ct']

## 2. 거래 기간 대비 총 거래 횟수

- Transaction_Frequency = Total_Trans_Ct/Months_on_book

In [6]:
df['Transaction_Frequency'] = df['Total_Trans_Ct']/df['Months_on_book']

## 3. 소득 대비 총 지불 금액

- Spending_Rate = Total_Trans_Amt/Approx_Income

In [7]:
income_map = {
    'Less than $40K': 20000,
    'Unknown': 60000, # Assuming average for unknowns
    '$40K - $60K': 50000,
    '$60K - $80K': 70000,
    '$80K - $120K': 100000,
    '$120K +': 160000
}
df['Approx_Income'] = df['Income_Category'].map(income_map)

df['Spending_Rate'] = df['Total_Trans_Amt'] / df['Approx_Income']

df.drop(['Approx_Income'], axis = 1, inplace = True)

## 4. 활동성 지수: 은행과의 상호작용 정도를 나타냄

- Activity_Index = {Total_Relationship_Count/max(Total_Relationship_Count)} + {1 - (Months_Inactive_12_mon/12)} + {Contacts_Count_12_mon/max(Contacts_Count_12_mon)} + {Total_Trans_Ct/max(Total_Trans_Ct)}

In [8]:
df['Activity_Index'] = (df['Total_Relationship_Count'] / df['Total_Relationship_Count'].max()  # 총 관계 수를 최대 관계 수로 나눈 비율
                        + (1 - df['Months_Inactive_12_mon'] / 12)  # 최근 12개월 동안 활동한 개월 수의 비율
                        + df['Contacts_Count_12_mon'] / df['Contacts_Count_12_mon'].max()  # 연락 횟수를 최대 연락 가능 횟수로 나눈 비율
                        + df['Total_Trans_Ct'] / df['Total_Trans_Ct'].max()) # 총 거래 횟수를 최대 거래 횟수로 나눈 비율


## 5. 평균 계좌 유지 기간 대비 활동 지수

- Engagement_Ratio = Activity_Index/Months_on_book

In [9]:
df['Engagement_Ratio'] = df['Activity_Index'] / df['Months_on_book']

## 6. 평균 지출 당 연체 비율

- Avg_Trans_Revolving_Ratio = Total_Revolving_Bal/Total_Trans_Amt

In [10]:
df['Avg_Trans_Revolving_Ratio'] = df['Total_Revolving_Bal'] / df['Total_Trans_Amt']  

## 6. Total_Trans_Amt 변수 변환

- 다봉 분포를 이루고 있어서 분포의 형태를 파악 후 그룹화 진행 위해 Clustering 진행

In [11]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
df['Total_Trans_Amt_Cluster'] = kmeans.fit_predict(df[['Total_Trans_Amt']])

C:\Users\82102\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Data save

In [12]:
df.to_csv('data_pre.csv', index=False, encoding='UTF-8-sig')

In [14]:
df

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Avg_Trans_Amt,Transaction_Frequency,Spending_Rate,Activity_Index,Engagement_Ratio,Avg_Trans_Revolving_Ratio,Total_Trans_Amt_Cluster
0,0,45,M,3,High School,Married,$60K - $80K,Blue,39,5,...,42,1.625,0.061,27.238095,1.076923,0.016343,2.552158,0.065440,0.679196,2
1,0,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,...,33,3.714,0.105,39.121212,0.750000,0.064550,2.487410,0.056532,0.669249,2
2,0,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,...,20,2.333,0.000,94.350000,0.555556,0.018870,1.727218,0.047978,0.000000,2
3,0,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,...,20,2.333,0.760,58.550000,0.588235,0.058550,1.477218,0.043448,2.149445,2
4,0,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,...,28,2.500,0.000,29.142857,1.333333,0.011657,1.951439,0.092926,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,0,50,M,2,Graduate,Single,$40K - $60K,Blue,40,3,...,117,0.857,0.462,132.273504,2.925000,0.309520,2.675060,0.066876,0.119605,1
10123,1,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,4,...,69,0.683,0.511,127.014493,2.760000,0.175280,2.496403,0.099856,0.249429,3
10124,1,44,F,1,High School,Married,Less than $40K,Blue,36,5,...,60,0.818,0.000,171.516667,1.666667,0.514550,2.681655,0.074490,0.000000,3
10125,1,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,4,...,62,0.722,0.000,135.403226,1.722222,0.167900,2.362710,0.065631,0.000000,3


## Box-Cox

In [13]:
#df['Total_Trans_Amt_transformed'], fitted_lambda = stats.boxcox(df['Total_Trans_Amt'])
#print("적용된 람다 값:", fitted_lambda)